## Example workflow to register multi-view light sheet data

Notes:
- install napari-stitcher for visualization functionality
- registration: so far only translation registration is performed
- fusion: only vanilla linear blending currently supported
- generally
  - this is a first hacky workflow that will change in API and become simplified
  - documentation will follow

In [1]:
# imports

import os
import numpy as np
from pathlib import Path
from tqdm import tqdm
import dask.diagnostics, tempfile

from ngff_stitcher import msi_utils, spatial_image_utils

%matplotlib notebook

# the package napari-stitcher is required for visualization with napari
VISUALIZE_USING_NAPARI = False

if VISUALIZE_USING_NAPARI:
    import napari
    from napari_stitcher import viewer_utils

2023-09-25 21:44:45,662 [Geometry3D WARNING] 'Renderer' is deprecated, using 'Visualizer' instead.


In [2]:
# Start a dask cluster

from distributed import Client, LocalCluster

lc = LocalCluster(n_workers=1, threads_per_worker=None)
client = Client(lc)
client


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 1
Total threads: 8,Total memory: 16.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:51107,Workers: 1
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 16.00 GiB
Comm: tcp://127.0.0.1:51113,Total threads: 8
Dashboard: http://127.0.0.1:51114/status,Memory: 16.00 GiB
Nanny: tcp://127.0.0.1:51110,


## Specify input data

In [3]:
base_dir = '../image-datasets/multi-view/old_mDSLM_classical_4_angles_10x_0.3NA_detection'
filenames = [(os.path.join(base_dir, f)) for f in os.listdir(base_dir) if f.endswith('.tif')]

# sort angles
filenames = [Path(fn) for fn in sorted(filenames)]
print('Files:')
print('\n'.join([fn.name for fn in filenames]))


Files:
MGolden2022A-DS0031TP0001DR0001CH0001PL(ZS).tif
MGolden2022A-DS0031TP0001DR0002CH0001PL(ZS).tif
MGolden2022A-DS0031TP0001DR0003CH0001PL(ZS).tif
MGolden2022A-DS0031TP0001DR0004CH0001PL(ZS).tif


## Convert input data to OME-Zarr

In [4]:
from ngff_stitcher import io

overwrite = False

msims = []
for filename in tqdm(filenames):
    store_path = filename.with_suffix('.zarr')
    if not os.path.exists(store_path) or overwrite:
        sim = io.read_tiff_into_spatial_xarray(
                        filename,
                        scale={'z': 2.58, 'y': 0.645, 'x': 0.645})
        msim = msi_utils.get_msim_from_sim(sim, scale_factors=None) # choose scale factors automatically
        msim.to_zarr(store_path)
    msim = msi_utils.multiscale_spatial_image_from_zarr(Path(store_path))
    msims.append(msim)

# alternative
# msims = []
# for filename in tqdm(filenames):
#     msim = msi_utils.get_store_decorator(
#         filename.with_suffix('.zarr'),
#         store_overwrite=False)(
#             msi_utils.get_msim_from_sim)(
#                 io.read_tiff_into_spatial_xarray(
#                     filename,
#                     scale={'z': 2.58, 'y': 0.645, 'x': 0.645}
#                 ))
#     msims.append(msim)


100%|██████████| 4/4 [00:00<00:00,  7.12it/s]


## Set estimate of initial transformations

In [5]:
import transformations as tf

for imsim, msim in enumerate(msims):

    # 90 degree rotation around x axis
    affine = tf.rotation_matrix(
        -np.pi/2 * imsim,
        point=spatial_image_utils.get_center_of_sim(msims[imsim]['scale0/image'], transform_key=None),
        direction=[0,0,1],
        )
    
    msi_utils.set_affine_transform(
        msim,
        affine[None], # one tp
        transform_key='affine_manual',
    )


### Visualize pre-registered views

In [6]:
if VISUALIZE_USING_NAPARI:

    viewer = napari.Viewer(ndisplay=3)
    lds = viewer_utils.create_image_layer_tuples_from_msims(
        msims, transform_key='affine_manual', n_colors=4, contrast_limits=[0, 1000])
    viewer_utils.add_image_layer_tuples_to_viewer(viewer, lds)
    viewer.reset_view()
    

/Users/malbert/mambaforge/envs/napari-stitcher/lib/python3.10/site-packages/napari/layers/image/image.py:761: UserWarning: Multiscale rendering is only supported in 2D. In 3D, only the lowest resolution scale is displayed
  warnings.warn(
/Users/malbert/mambaforge/envs/napari-stitcher/lib/python3.10/site-packages/napari/layers/image/image.py:761: UserWarning: Multiscale rendering is only supported in 2D. In 3D, only the lowest resolution scale is displayed
  warnings.warn(
/Users/malbert/mambaforge/envs/napari-stitcher/lib/python3.10/site-packages/napari/layers/utils/_slice_input.py:72: UserWarning: Non-orthogonal slicing is being requested, but is not fully supported. Data is displayed without applying an out-of-slice rotation or shear component.
  warnings.warn(
/Users/malbert/mambaforge/envs/napari-stitcher/lib/python3.10/site-packages/napari/layers/image/image.py:761: UserWarning: Multiscale rendering is only supported in 2D. In 3D, only the lowest resolution scale is displayed
  w

### Register views

In [12]:
# import skimage.registration
from ngff_stitcher import registration

with dask.diagnostics.ProgressBar():

    params = registration.register(
        # [msi_utils.get_sim_from_msim(msim) for msim in msims],
        msims,
        registration_binning={'z': 2, 'y': 8, 'x': 8},
        reg_channel_index=0,
        transform_key='affine_manual',
    )
    
for msim, param in zip(msims, params):
    msi_utils.set_affine_transform(msim, param, transform_key='affine_registered', base_transform_key='affine_manual')

hello55
hello55
hello55
hello55
hello55
hello55
hello55
hello55
hello55
hello55
hello55
hello55
hello55
hello55
hello55
hello55
hello55
hello55
hello55
hello55
hello55
hello55
hello55
hello55


### Visualize registration

In [8]:
if VISUALIZE_USING_NAPARI:
    
    viewer = napari.Viewer(ndisplay=3)

    lds = viewer_utils.create_image_layer_tuples_from_msims(
        msims, transform_key='affine_manual', n_colors=4,
        name_prefix='pre-registered view',
        contrast_limits=[0, 1000],
        )
    mlayers = viewer_utils.add_image_layer_tuples_to_viewer(viewer, lds, do_link_layers=True)

    lds = viewer_utils.create_image_layer_tuples_from_msims(
        msims, transform_key='affine_registered', n_colors=4,
        name_prefix='registered view',
        contrast_limits=[0, 1000],
        )
    rlayers = viewer_utils.add_image_layer_tuples_to_viewer(viewer, lds, do_link_layers=True)

/Users/malbert/mambaforge/envs/napari-stitcher/lib/python3.10/site-packages/napari/layers/image/image.py:761: UserWarning: Multiscale rendering is only supported in 2D. In 3D, only the lowest resolution scale is displayed
  warnings.warn(
/Users/malbert/mambaforge/envs/napari-stitcher/lib/python3.10/site-packages/napari/layers/image/image.py:761: UserWarning: Multiscale rendering is only supported in 2D. In 3D, only the lowest resolution scale is displayed
  warnings.warn(
/Users/malbert/mambaforge/envs/napari-stitcher/lib/python3.10/site-packages/napari/layers/utils/_slice_input.py:72: UserWarning: Non-orthogonal slicing is being requested, but is not fully supported. Data is displayed without applying an out-of-slice rotation or shear component.
  warnings.warn(
/Users/malbert/mambaforge/envs/napari-stitcher/lib/python3.10/site-packages/napari/layers/image/image.py:761: UserWarning: Multiscale rendering is only supported in 2D. In 3D, only the lowest resolution scale is displayed
  w

### Fuse views (linear blending)

In [9]:
from ngff_stitcher import fusion
import dask.array as da

sims = [msi_utils.get_sim_from_msim(msim) for msim in msims]

tmpdir = tempfile.TemporaryDirectory()

fused = fusion.fuse(
    sims[:],
    transform_key='affine_registered',
    output_spacing={dim: 5. for dim in ['z', 'y', 'x']},
    output_chunksize=128,
    )

print('Fusing views...')
with dask.diagnostics.ProgressBar():

    fused.data = da.to_zarr(
        fused.data,
        os.path.join(tmpdir.name, 'fused_sim.zarr'),
        overwrite=True, return_stored=True)

print('Creating multiscale output OME-Zarr...')
with dask.diagnostics.ProgressBar():

    mfused = msi_utils.get_msim_from_sim(fused, scale_factors=None)

    fused_path = os.path.join(tmpdir.name, 'fused.zarr')
    mfused.to_zarr(fused_path)
    
mfused = msi_utils.multiscale_spatial_image_from_zarr(fused_path)

Fusing views...


/Users/malbert/mambaforge/envs/napari-stitcher/lib/python3.10/site-packages/distributed/client.py:3149: UserWarning: Sending large graph of size 0.97 GiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


Creating multiscale output OME-Zarr...


### Visualize fusion in napari

In [10]:
if VISUALIZE_USING_NAPARI:

    viewer = napari.Viewer(ndisplay=3)

    lds = viewer_utils.create_image_layer_tuples_from_msims(
        msims, transform_key='affine_registered', n_colors=4,
        name_prefix='registered view',
        contrast_limits=[0, 1000]
        )

    rlayers = viewer_utils.add_image_layer_tuples_to_viewer(
        viewer, lds, do_link_layers=False)

    lds = viewer_utils.create_image_layer_tuples_from_msim(
        mfused,
        transform_key='affine_registered',
        name_prefix='fused',
        contrast_limits=[0, 1000])

    viewer_utils.add_image_layer_tuples_to_viewer(viewer, lds)

/Users/malbert/mambaforge/envs/napari-stitcher/lib/python3.10/site-packages/napari/layers/image/image.py:761: UserWarning: Multiscale rendering is only supported in 2D. In 3D, only the lowest resolution scale is displayed
  warnings.warn(
/Users/malbert/mambaforge/envs/napari-stitcher/lib/python3.10/site-packages/napari/layers/image/image.py:761: UserWarning: Multiscale rendering is only supported in 2D. In 3D, only the lowest resolution scale is displayed
  warnings.warn(
/Users/malbert/mambaforge/envs/napari-stitcher/lib/python3.10/site-packages/napari/layers/utils/_slice_input.py:72: UserWarning: Non-orthogonal slicing is being requested, but is not fully supported. Data is displayed without applying an out-of-slice rotation or shear component.
  warnings.warn(
/Users/malbert/mambaforge/envs/napari-stitcher/lib/python3.10/site-packages/napari/layers/image/image.py:761: UserWarning: Multiscale rendering is only supported in 2D. In 3D, only the lowest resolution scale is displayed
  w

In [11]:
# stream presaved fused image to tif

from ngff_stitcher import io

with dask.diagnostics.ProgressBar():
    io.save_sim_as_tif('fused.tif', msi_utils.get_sim_from_msim(mfused))

[########################################] | 100% Completed | 107.39 ms


/Users/malbert/mambaforge/envs/napari-stitcher/lib/python3.10/site-packages/dask/base.py:1402: UserWarning: Running on a single-machine scheduler when a distributed client is active might lead to unexpected results.
  warnings.warn(
